In [1]:
%matplotlib widget

from pathlib import Path
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import svd
import imageio
from scipy import ndimage

import h5py
import stempy.io as stio
import stempy.image as stim

# Set up Cori paths
ncemhub = Path('/global/cfs/cdirs/ncemhub/4Dcamera/')
scratch = Path('/global/cscratch1/sd/percius/')
# Set up mothership6 paths
hdd1 = Path('/mnt/hdd1')

In [30]:
def com_sparse_iterative(electron_events, scan_dimensions, crop_to=(576,576)):
    # Iterative version. Will be replaced in a future stempy release
    # Calculate the center of mass as a test
    com2 = np.zeros((2, scan_dimensions[0]*scan_dimensions[1]), np.float32)
    for ii, ev in enumerate(electron_events):
        if len(ev) > 0:
            x,y = np.unravel_index(ev,(576,576))
            mm = len(ev)
            comx0 = np.sum(x) / mm
            comy0 = np.sum(y) / mm
            # Crop around the center
            keep = (x > comx0-crop_to[0]) & (x <= comx0+crop_to[0]) & (y > comy0-crop_to[1]) & (y <= comy0+crop_to[1])
            x = x[keep]
            y = y[keep]
            mm = len(x)
            if mm > 0:
                comx = np.sum(x)
                comy = np.sum(y)
                comx = comx / mm
                comy = comy / mm
            else:
                comx = comx0
                comy = comy0
            com2[:, ii] = (comy,comx)
        else:
            com2[:, ii] = (np.nan, np.nan)
            
    com2 = com2.reshape((2, scan_dimensions[0], scan_dimensions[1]))
    return com2

def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.

    """

    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

In [3]:
# Close all previous windows to avoid too many windows
plt.close('all')

# Load a sparse vacuum 4D camera data set 
scan_num = 105
threshold = 4.0
data_dir = Path('2020.11.23')

fname = hdd1 / data_dir / Path('data_scan{}_th{}_electrons.h5'.format(scan_num, threshold))

vacuum_scan = stio.load_electron_counts(fname)

print('File: {}'.format(fname))
print('Initial scan dimensions = {}'.format(vacuum_scan.scan_dimensions))

File: /mnt/hdd1/2020.11.23/data_scan105_th4.0_electrons.h5
Initial scan dimensions = [257, 1024]


In [5]:
# Show the summed diffraction pattern
dp = stim.calculate_sum_sparse(vacuum_scan.data, vacuum_scan.frame_dimensions)

fg,ax = plt.subplots(1,1)
ax.imshow(dp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
# Calculate the com iteratively
#com2 = stim.com_sparse(vacuum_scan.data, vacuum_scan.frame_dimensions)
com2 = com_sparse_iterative(vacuum_scan.data, vacuum_scan.scan_dimensions, crop_to=(30, 30))

# These will be removed in a future release
print('Remove the code below in a future release.')
# Nan values to average value
np.nan_to_num(com2[0,],copy=False,nan=np.nanmean(com2[0,]))
np.nan_to_num(com2[1,],copy=False,nan=np.nanmean(com2[1,]));

com2 = com2.reshape((2,*vacuum_scan.scan_dimensions[::-1]))


Remove the code below in a future release.


In [32]:
# Remove the outliers by median filtering
com2_filt = np.zeros_like(com2)
com2_filt[0,] = ndimage.median_filter(com2[0,], size=(3,3))
com2_filt[1,] = ndimage.median_filter(com2[1,], size=(3,3))

com2_median = np.median(com2_filt,axis=(1,2))

fg,ax = plt.subplots(1, 2,sharex=True,sharey=True)
ax[0].imshow(com2_filt[0,]-com2_median[0],cmap='bwr',vmin=-25,vmax=25)
ax[1].imshow(com2_filt[1,]-com2_median[1],cmap='bwr',vmin=-25,vmax=25)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# Fit the COMs to planes to smooth it out
YY, XX = np.mgrid[0:com2.shape[1],0:com2.shape[2]]

planeCOM0 = planeFit(np.stack((YY,XX,com2_filt[0,])))
planeCOM1 = planeFit(np.stack((YY,XX,com2_filt[1,])))

print(planeCOM0)
print(planeCOM1)

(array([511.5       , 128.        , 281.50863554]), array([-0.01335387, -0.01116126,  0.99984854]))
(array([511.5      , 128.       , 296.8526089]), array([0.01876338, 0.03832418, 0.99908918]))


In [33]:
# Generate points on the plane to fit the dataset size
YY, XX = np.mgrid[0:vacuum_scan.scan_dimensions[1], 0:vacuum_scan.scan_dimensions[0]]

normal = planeCOM0[1]
d = np.dot(-planeCOM0[0], normal)
# calculate corresponding z
z0 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

normal = planeCOM1[1]
d = np.dot(-planeCOM1[0], normal)
# calculate corresponding z
z1 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

fg,ax = plt.subplots(2,2)
ax[0,0].imshow(com2_filt[0,],cmap='bwr')
ax[0,1].imshow(z0, cmap='bwr')
ax[1,0].imshow(com2_filt[1,],cmap='bwr')
ax[1,1].imshow(z1, cmap='bwr');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# Test centering on the vacuum scan itself
vacuum_scan_centered = namedtuple('ElectronCountedData',
                             ['data', 'scan_dimensions', 'frame_dimensions'])
vacuum_scan_centered.scan_dimensions = vacuum_scan.scan_dimensions
vacuum_scan_centered.frame_dimensions = vacuum_scan.frame_dimensions

vacuum_scan_centered.data = []

z0_round = np.round(z0).astype(np.int32) - int(z0.mean())
z1_round = np.round(z1).astype(np.int32) - int(z1.mean())

for ev, x, y in zip(vacuum_scan.data, z0_round.ravel(), z1_round.ravel()):
    evx, evy = np.unravel_index(ev, (576,576))
    evx_centered = evx - y
    evy_centered = evy - x
    
    keep = (evx_centered < 576) & (evx_centered >= 0) * (evy_centered < 576) & (evy_centered >= 0)
    evx_centered = evx_centered[keep]
    evy_centered = evy_centered[keep]
    
    vacuum_scan_centered.data.append(np.ravel_multi_index((evx_centered,evy_centered), (576,576)))
vacuum_scan_centered.data = np.array(vacuum_scan_centered.data, dtype=object)

dp = stim.calculate_sum_sparse(vacuum_scan.data, vacuum_scan.frame_dimensions)
    
dp2 = stim.calculate_sum_sparse(vacuum_scan_centered.data, vacuum_scan_centered.frame_dimensions)

fg,ax = plt.subplots(1,2,sharex=True,sharey=True)
ax[0].imshow(dp)
ax[1].imshow(dp2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
# Compare com_filtered to plane fit
# Nan values to average value
np.nan_to_num(com2[0,],copy=False,nan=np.nanmean(com2[0,]))
np.nan_to_num(com2[1,],copy=False,nan=np.nanmean(com2[1,]))

fg,ax = plt.subplots(2,2)
ax[0,0].imshow(z0,cmap='bwr')
ax[0,1].imshow(z1,cmap='bwr')
ax[1,0].imshow(com2[0,]-z0,cmap='bwr')
ax[1,1].imshow(com2[1,]-z1,cmap='bwr')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Apply to experiment from a sample

In [36]:
# Load a sparse 4D camera data set
scan_num =102
threshold = 4.0
data_dir = Path('2020.11.23')

fname = hdd1 / data_dir / Path('data_scan{}_th{}_electrons.h5'.format(scan_num, threshold))
#fname = Path.home() / Path('data/temp/data_scan{scan_num}_th{}_electrons.h5'.format(scan_num, threshold))

experiment = stio.load_electron_counts(fname)

print('File: {}'.format(fname))
print('Initial scan dimensions = {}'.format(experiment.scan_dimensions))

File: /mnt/hdd1/2020.11.23/data_scan102_th4.0_electrons.h5
Initial scan dimensions = [257, 1024]


In [37]:
# Generate points on the plane to fit the dataset size
factor = (experiment.scan_dimensions[0] / vacuum_scan.scan_dimensions[0],
         experiment.scan_dimensions[1] / vacuum_scan.scan_dimensions[1])

# Generate positions between vacuum positions
YY, XX = np.mgrid[0:experiment.scan_dimensions[0], 0:experiment.scan_dimensions[1]]
YY = YY.astype('<f4') / factor[1]
XX = XX.astype('<f4') / factor[0]

normal = planeCOM0[1]
d = np.dot(-planeCOM0[0], normal)
# calculate corresponding z
z0 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

normal = planeCOM1[1]
d = np.dot(-planeCOM1[0], normal)
# calculate corresponding z
z1 = (-normal[0]*YY - normal[1]*XX - d)/normal[2]

# Round to integers
z0_round = np.round(z0 - z0.mean()).astype(np.int64)
z1_round = np.round(z1 - z1.mean()).astype(np.int64)

fg,ax = plt.subplots(2,2)
ax[0,0].imshow(z0,cmap='bwr')
ax[0,1].imshow(z0_round, cmap='bwr')
ax[1,0].imshow(z1,cmap='bwr')
ax[1,1].imshow(z1_round, cmap='bwr');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
# Use the fitted plane from the vacuum scan to recenter the events
scan_centered = []

for ev, x, y in zip(experiment.data, z0_round.ravel(), z1_round.ravel()):
    evx, evy = np.unravel_index(ev, (576,576))
    evx_centered = evx - y # need to flip x and y
    evy_centered = evy - x
    
    # Some events will get pushed off the detetor by the shift. Remove them
    keep = (evx_centered < 576) & (evx_centered >= 0) & (evy_centered < 576) & (evy_centered >= 0)
    evx_centered = evx_centered[keep]
    evy_centered = evy_centered[keep]
    
    scan_centered.append(np.ravel_multi_index((evx_centered,evy_centered), (576,576)))
scan_centered = np.array(scan_centered, dtype=object)

# Create a stempy counted data namedtuple
experiment_centered = namedtuple('ElectronCountedData',
                             ['data', 'scan_dimensions', 'frame_dimensions'])
experiment_centered.data = scan_centered
experiment_centered.scan_dimensions = experiment.scan_dimensions[::1]
experiment_centered.frame_dimensions = experiment.frame_dimensions

dp = stim.calculate_sum_sparse(experiment.data, experiment.frame_dimensions)
dp2 = stim.calculate_sum_sparse(experiment_centered.data, experiment_centered.frame_dimensions)

fg,ax = plt.subplots(2,1,sharex=True,sharey=True)
ax[0].imshow(dp)
ax[1].imshow(np.log(dp2+0.1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
# Save to a stempy dataset
out_name = fname.with_name('data_scan{}_th{}_electrons_centered.h5'.format(scan_num,threshold))
stio.save_electron_counts(out_name,experiment_centered)
print(out_name)

/mnt/hdd1/2020.11.23/temp.h5
